

# Web Scraper
***
This web scraper is designed to scrape web content where the content is both static and dynamic (generated by Javascrpt) or more specifically, where parts of the content (user comments) are managed by disqus.com.

Prerequisites:
- install selenium (pip install selenium)
- install BeautifulSoup (pip install BeautifulSoup4)
- download and install Chrome web browser
***

### Import the required libraries

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import json
import csv
import os.path

### Set up the web driver

In [2]:
# Input for selenium (for making http requests)
driver_path = './chromedriver'
s = Service(driver_path)
driver = webdriver.Chrome(service=s)

### Open the page in Chrome browser

In [3]:
# URL of the web page for scraping
#url = 'https://teslamag.de/news/supercharger-limburg-tesla-karte-geoffnet-fremde-elektroautos-49280'
#url = 'https://teslamag.de/news/test-niederlande-tesla-oeffnet-erste-zehn-supercharger-fremde-elektroautos-42542'
#url = 'https://teslamag.de/news/fremde-elektroautos-supercharger-vw-id3-problemlos-audi-fahrer-ueberzeugt-42649'
url = 'https://teslamag.de/news/kreativ-supercharger-tesla-andere-elektroauto-fahrer-loesungen-teilen-49209'

driver.get(url)

In [4]:
page_html = driver.page_source
soup = BeautifulSoup(page_html, "html.parser")

In [41]:
# Optional - show the result
#print(soup.title.text)
#print(soup.prettify())

### Get the Javascript code that generates the comments (via disqus.com)

In [5]:
disqus = soup.find_all('div', id='disqus_thread')
post_js_url = disqus[0].find('iframe').attrs['src']

In [6]:
# Optional - print the url that makes the disqus request 
#print(comments_js_url)

### Get the comments (from Disqus)

- Get and parse the posts from disqus javascript

In [6]:
driver.get(post_js_url)
post_html = driver.page_source
soup2 = BeautifulSoup(post_html, "html.parser")

- Get the thread data from the script tag & convert it into a dictionary

In [8]:
thread_data = soup2.find('script', id='disqus-threadData')
posts = json.loads(thread_data.string)
posts = posts['response']['posts']

- Save comments to csv

In [34]:
csv_file_path = './comments.csv'
new_file = False
if not os.path.exists('./comments.csv'):
    new_file = True

with open(csv_file_path, 'a', encoding='UTF8') as f:
    writer = csv.writer(f)
    if new_file:
        header = ['article', 'comment', 'created', 'likes', 'dislikes']
        writer.writerow(header)
    
    for post in posts:
        writer.writerow([soup.title.text, post['raw_message'], post['createdAt'], post['likes'], post['dislikes']])

True
